In [26]:
import sys
sys.path.append('../')

from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.trading_strategy.pivot_strategy import *

In [27]:
dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'CLOSE', 'LOW', 'TRANSACTIONS','HIGHT'])

data = dataloader.load_dataset({'AAPL':'../data/cryptos/BTCUSD_2020-04-07_2022-04-06.parquet'})
data

,AAPL_VOLUME,AAPL_OPEN,AAPL_CLOSE,AAPL_LOW,AAPL_TRANSACTIONS,AAPL_HIGHT
DATE,,,,,,
2020-04-07 17:48:00,15.835240,7363.09,7352.66,7343.13,101,7364.21
2020-04-07 17:49:00,7.652123,7358.42,7361.54,7349.32,69,7367.90
2020-04-07 17:50:00,16.446123,7361.54,7354.09,7340.09,93,7361.54
2020-04-07 17:51:00,7.068813,7354.14,7345.00,7340.00,71,7357.53
2020-04-07 17:52:00,8.162850,7350.10,7355.16,7343.35,71,7357.00
...,...,...,...,...,...,...
2022-04-06 23:54:00,37.006803,43146.71,43144.08,43123.70,683,43187.00
2022-04-06 23:55:00,65.685080,43144.08,43188.59,43138.37,1232,43261.00
2022-04-06 23:56:00,25.763792,43188.44,43111.80,43111.80,797,43233.00


In [28]:
resampled_data_5min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'5min')
resampled_data_15min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'15min')
resampled_data_30min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'30min')
resampled_data_1hr = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1h')
resampled_data_1d = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1d')
resampled_data_1wk = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1W')

D:\FAU\Ml in finance_ex\github_repo\notebooks\..\src\finance_ml\trading_strategy\pivot_strategy.py:82: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  resampled_data.index = resampled_data.index + pd.DateOffset(weekday=0)


In [29]:
traditional_pivot_1d = traditional_pivot(resampled_data_1d,'AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE')
traditional_pivot_1wk = traditional_pivot(resampled_data_1wk,'AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE')

In [30]:
traditional_pivot_1d

,DATE,Pivot,S1,S2,S3,R1,R2,R3
1,2020-04-08,7213.843333,7056.286667,6915.443333,6757.886667,7354.686667,7512.243333,7653.086667
2,2020-04-09,7314.214971,7198.429941,7035.859883,6920.074853,7476.785029,7592.570059,7755.140117
3,2020-04-10,7254.700000,7131.300000,6968.500000,6845.100000,7417.500000,7540.900000,7703.700000
4,2020-04-11,6973.630000,6641.160000,6408.520000,6076.050000,7206.270000,7538.740000,7771.380000
5,2020-04-12,6870.106667,6780.213333,6670.826667,6580.933333,6979.493333,7069.386667,7178.773333
...,...,...,...,...,...,...,...,...
725,2022-04-02,45756.635083,44772.564917,43248.789833,42264.719667,47280.410167,48264.480333,49788.255417
726,2022-04-03,46234.090000,45228.180000,44630.090000,43624.180000,46832.180000,47838.090000,48436.180000
727,2022-04-04,46509.053333,45418.106667,44414.053333,43323.106667,47513.106667,48604.053333,49608.106667
728,2022-04-05,46202.610000,45505.220000,44413.610000,43716.220000,47294.220000,47991.610000,49083.220000


In [31]:
mereged_data = merge_ohlc_pivot(resampled_data_15min,traditional_pivot_1d)

In [32]:
traded_buy = generate_buy(mereged_data,'AAPL_CLOSE','AAPL_OPEN')

In [33]:
traded_sell = generate_sell(mereged_data,'AAPL_CLOSE','AAPL_OPEN')

In [34]:
pnl_buy = get_pnl(traded_buy,'AAPL_CLOSE')

In [35]:
pnl_sell = get_pnl(traded_sell,'AAPL_CLOSE')

In [36]:
traded_buy.to_excel('test_buy.xlsx', index=False)

In [37]:
traded_sell.to_excel('test_sell.xlsx', index=False)

In [42]:
type(traded_sell)

pandas.core.frame.DataFrame

In [47]:
m1 = pd.concat([traded_buy,traded_sell], axis=0)

In [48]:
m1.to_excel('merge.xlsx')